<h2>LSTM</h2>

In [6]:
import nltk
import json
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Load intents
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Extract text and labels from intents
texts = []
labels = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        texts.append(pattern)
        labels.append(intent['tag'])

# Tokenize the text data
max_words = 1000
tokenizer = Tokenizer(num_words=max_words, lower=True, split=" ")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

# Pad sequences
max_sequence_length = max([len(seq) for seq in sequences])
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
num_classes = len(le.classes_)

# Define the LSTM model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_sequence_length))
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(LSTM(64, kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.001)))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
#model.fit(data, encoded_labels, validation_split=0.1, epochs=100, batch_size=8, callbacks=[early_stopping])
model.fit(data, encoded_labels, epochs=100, batch_size=8, verbose=1)


Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


109/109 [==============================] - 10s 50ms/step - loss: 3.8178 - accuracy: 0.1010
Epoch 2/100
109/109 [==============================] - 5s 43ms/step - loss: 3.6142 - accuracy: 0.0999
Epoch 3/100
109/109 [==============================] - 6s 52ms/step - loss: 3.3534 - accuracy: 0.1056
Epoch 4/100
109/109 [==============================] - 5s 42ms/step - loss: 3.1931 - accuracy: 0.1515
Epoch 5/100
109/109 [==============================] - 5s 42ms/step - loss: 3.0256 - accuracy: 0.1894
Epoch 6/100
109/109 [==============================] - 6s 53ms/step - loss: 2.8838 - accuracy: 0.2158
Epoch 7/100
109/109 [==============================] - 5s 42ms/step - loss: 2.6642 - accuracy: 0.2939
Epoch 8/100
109/109 [==============================] - 6s 53ms/step - loss: 2.4876 - accuracy: 0.3364
Epoch 9/100
109/109 [==============================] - 5s 42ms/step - loss: 2.3372 - accuracy: 0.3594
Epoch 10/100
109/109 [==============================] - 5s 43ms/step - loss: 2.2348 - accurac

In [7]:
# Load testing intents
data_file_test = open('intents_testing.json').read()
intents_test = json.loads(data_file_test)

# Extract text and labels from testing intents
texts_test = []
labels_test = []

for intent in intents_test['intents']:
    for pattern in intent['patterns']:
        texts_test.append(pattern)
        labels_test.append(intent['tag'])

# Tokenize and pad the testing text data
sequences_test = tokenizer.texts_to_sequences(texts_test)
data_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Encode testing labels
encoded_labels_test = le.transform(labels_test)

# Evaluate the model on the training set
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print('Train accuracy:', train_acc)

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(data_test, encoded_labels_test, verbose=0)
print('Test accuracy:', test_acc)

Train accuracy: 0.9565772414207458
Test accuracy: 0.5921052694320679
